# 04 - Test and Deploy Training Pipeline to Vertex Pipelines

The purpose of this notebook is to test, deploy, and run the `TFX` pipeline on `Vertex Pipelines`. The notebook covers the following tasks:
1. Run the tests locally.
2. Run the pipeline using `Vertex Pipelines`
3. Execute the pipeline deployment `CI/CD` steps using `Cloud Build`.

## Setup

### Import libraries

In [1]:
import os
import kfp
import tfx

print("Tensorflow Version:", tfx.__version__)
print("KFP Version:", kfp.__version__)

Tensorflow Version: 1.8.0
KFP Version: 1.8.12


### Setup Google Cloud project

In [2]:
PROJECT = 'mwpmltr' # Change to your project id.
REGION = 'us-central1' # Change to your region.
BUCKET = 'gcp-certification-chicago-taxi-demo' # Change to your bucket name.
SERVICE_ACCOUNT = "55590906972-compute@developer.gserviceaccount.com"

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP project id
    BUCKET = PROJECT
    # Try to create the bucket if it doesn't exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)
print("Service Account:", SERVICE_ACCOUNT)

Project ID: mwpmltr
Region: us-central1
Bucket name: gcp-certification-chicago-taxi-demo
Service Account: 55590906972-compute@developer.gserviceaccount.com


In [3]:
!gcloud config set account $SERVICE_ACCOUNT

Updated property [core/account].


### Set configurations

In [4]:
BQ_LOCATION = 'us-central1'
BQ_DATASET_NAME = 'playground_central' # Change to your BQ dataset name.
BQ_TABLE_NAME = 'chicago_taxitrips_prep'

VERSION = 'v01'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT}/{CICD_IMAGE_NAME}"

In [5]:
!rm -r src/raw_schema/.ipynb_checkpoints/

rm: cannot remove 'src/raw_schema/.ipynb_checkpoints/': No such file or directory


In [6]:
# Fix this error:
# ImportError: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.30' not found
LD_LIBRARY_PATH = f'/opt/conda/lib/'
os.environ["LD_LIBRARY_PATH"] = LD_LIBRARY_PATH

## 1. Run the CICD steps locally

### Set pipeline configurations for the local run

In [7]:
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"] =  MODEL_DISPLAY_NAME
os.environ["PIPELINE_NAME"] = PIPELINE_NAME
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["BQ_LOCATION"] = BQ_LOCATION
os.environ["BQ_DATASET_NAME"] = BQ_DATASET_NAME
os.environ["BQ_TABLE_NAME"] = BQ_TABLE_NAME
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/e2e_tests"
os.environ["TRAIN_LIMIT"] = "1000"
os.environ["TEST_LIMIT"] = "100"
os.environ["UPLOAD_MODEL"] = "0"
os.environ["ACCURACY_THRESHOLD"] = "0.1"
os.environ["BEAM_RUNNER"] = "DirectRunner"
os.environ["TRAINING_RUNNER"] = "local"

In [8]:
from src.tfx_pipelines import config
import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

2022-09-10 23:46:28.076616: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/
2022-09-10 23:46:28.076660: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


PROJECT: mwpmltr
REGION: us-central1
GCS_LOCATION: gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/e2e_tests
ARTIFACT_STORE_URI: gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/e2e_tests/tfx_artifacts
MODEL_REGISTRY_URI: gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/e2e_tests/model_registry
DATASET_DISPLAY_NAME: chicago-taxi-tips
MODEL_DISPLAY_NAME: chicago-taxi-tips-classifier-v01
PIPELINE_NAME: chicago-taxi-tips-classifier-v01-train-pipeline
ML_USE_COLUMN: ml_use
EXCLUDE_COLUMNS: trip_start_timestamp
TRAIN_LIMIT: 1000
TEST_LIMIT: 100
SERVE_LIMIT: 0
NUM_TRAIN_SPLITS: 4
NUM_EVAL_SPLITS: 1
ACCURACY_THRESHOLD: 0.1
USE_KFP_SA: False
TFX_IMAGE_URI: gcr.io/mwpmltr/tfx-chicago-taxi-tips:latest
BEAM_RUNNER: DirectRunner
BEAM_DIRECT_PIPELINE_ARGS: ['--project=mwpmltr', '--temp_location=gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/e2e_tests/temp']
BEAM_DATAFLOW_PIPELINE_ARGS: ['--project=mwpmltr', '--temp_location=gs://gcp-certification-chicago-t

### Run unit tests

In [9]:
!pytest src/tests/datasource_utils_tests.py -s

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/jupyter/mlops-with-vertex-ai
plugins: anyio-3.6.1
collected 2 items                                                              

src/tests/datasource_utils_tests.py BigQuery Source: mwpmltr.playground_central.chicago_taxitrips_prep
.BigQuery Source: mwpmltr.playground_central.chicago_taxitrips_prep
.

============================== 2 passed in 4.71s ===============================


In [10]:
!pytest src/tests/model_tests.py --disable-warnings

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/jupyter/mlops-with-vertex-ai
plugins: anyio-3.6.1
collected 2 items                                                              

src/tests/model_tests.py ..                                              [100%]

======================== 2 passed, 10 warnings in 2.70s ========================


### Run e2e pipeline test

In [11]:
!pytest src/tests/pipeline_deployment_tests.py::test_e2e_pipeline -s

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.2, pluggy-1.0.0
rootdir: /home/jupyter/mlops-with-vertex-ai
plugins: anyio-3.6.1
collecting ... 2022-09-10 23:46:42.185084: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/
2022-09-10 23:46:42.185136: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
collected 1 item                                                               

src/tests/pipeline_deployment_tests.py upload_model: 0
Pipeline e2e test artifacts stored in: gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/e2e_tests
ML metadata store is ready.
Excluding no splits because exclude_splits is not set.
Excluding no splits because ex

## 2. Run the training pipeline using Vertex Pipelines



### Set the pipeline configurations for the Vertex AI run

In [12]:
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"] = MODEL_DISPLAY_NAME
os.environ["PIPELINE_NAME"] = PIPELINE_NAME
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}"
os.environ["TRAIN_LIMIT"] = "85000"
os.environ["TEST_LIMIT"] = "15000"
os.environ["BEAM_RUNNER"] = "DataflowRunner"
os.environ["TRAINING_RUNNER"] = "vertex"
os.environ["TFX_IMAGE_URI"] = f"gcr.io/{PROJECT}/{DATASET_DISPLAY_NAME}:{VERSION}"
os.environ["ENABLE_CACHE"] = "1"

# Debug
os.environ["SERVING_RUNTIME"] = "tf2-cpu.2-8"
# os.environ["UPLOAD_MODEL"] = "1" # # testing purposes: why was this missing?

In [13]:
from src.tfx_pipelines import config
import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

PROJECT: mwpmltr
REGION: us-central1
GCS_LOCATION: gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips
ARTIFACT_STORE_URI: gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/tfx_artifacts
MODEL_REGISTRY_URI: gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/e2e_tests/model_registry
DATASET_DISPLAY_NAME: chicago-taxi-tips
MODEL_DISPLAY_NAME: chicago-taxi-tips-classifier-v01
PIPELINE_NAME: chicago-taxi-tips-classifier-v01-train-pipeline
ML_USE_COLUMN: ml_use
EXCLUDE_COLUMNS: trip_start_timestamp
TRAIN_LIMIT: 85000
TEST_LIMIT: 15000
SERVE_LIMIT: 0
NUM_TRAIN_SPLITS: 4
NUM_EVAL_SPLITS: 1
ACCURACY_THRESHOLD: 0.1
USE_KFP_SA: False
TFX_IMAGE_URI: gcr.io/mwpmltr/chicago-taxi-tips:v01
BEAM_RUNNER: DataflowRunner
BEAM_DIRECT_PIPELINE_ARGS: ['--project=mwpmltr', '--temp_location=gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/temp']
BEAM_DATAFLOW_PIPELINE_ARGS: ['--project=mwpmltr', '--temp_location=gs://gcp-certification-chicago-taxi-demo/chicago-taxi-tips/temp'

### Build the ML container image

This is the `TFX` runtime environment for the training pipeline steps.

In [14]:
!echo $TFX_IMAGE_URI

gcr.io/mwpmltr/chicago-taxi-tips:v01


In [15]:
!gcloud builds submit --tag $TFX_IMAGE_URI . --timeout=15m --machine-type=e2-highcpu-8

Creating temporary tarball archive of 59 file(s) totalling 1.5 MiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/home/jupyter/.config/gcloud/logs/2022.09.10/23.49.59.476470.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [.] to [gs://mwpmltr_cloudbuild/source/1662853799.572005-91114e838bc44ba690f6da0299d3603c.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/mwpmltr/locations/global/builds/bea39746-48aa-4fde-8d02-21d26022dfba].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/bea39746-48aa-4fde-8d02-21d26022dfba?project=55590906972].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "bea39746-48aa-4fde-8d02-21d26022dfba"

FETCHSOURCE
Fetching storage object: gs://mwpmltr_cloudbuild/source/1662853799.572005-91114e838bc44ba690f6da0299d3603c.tgz#166285380013

### Compile pipeline

In [16]:
from src.tfx_pipelines import runner

pipeline_definition_file = f'{config.PIPELINE_NAME}.json'
pipeline_definition = runner.compile_training_pipeline(pipeline_definition_file)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying etl.py -> build/lib
copying transformations.py -> build/lib
installing to /tmp/tmpyay0iirs
running install
running install_lib
copying build/lib/transformations.py -> /tmp/tmpyay0iirs
copying build/lib/etl.py -> /tmp/tmpyay0iirs
running install_egg_info
running egg_info
creating tfx_user_code_DataTransformer.egg-info
writing tfx_user_code_DataTransformer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_DataTransformer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_DataTransformer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_DataTransformer.egg-info/SOURCES.txt'
Copying tfx_user_code_DataTransformer.egg-info to /tmp/tmpyay0iirs/tfx_user_code_DataTransformer-0.0+de07c8431e7a29dced215501daf4f187c64541

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying data.py -> build/lib
copying exporter.py -> build/lib
copying trainer.py -> build/lib
copying task.py -> build/lib
copying model.py -> build/lib
copying runner.py -> build/lib
copying defaults.py -> build/lib
installing to /tmp/tmp7f9yf1yi
running install
running install_lib
copying build/lib/trainer.py -> /tmp/tmp7f9yf1yi
copying build/lib/model.py -> /tmp/tmp7f9yf1yi
copying build/lib/runner.py -> /tmp/tmp7f9yf1yi
copying build/lib/task.py -> /tmp/tmp7f9yf1yi
copying build/lib/data.py -> /tmp/tmp7f9yf1yi
copying build/lib/defaults.py -> /tmp/tmp7f9yf1yi
copying build/lib/exporter.py -> /tmp/tmp7f9yf1yi
running install_egg_info
running egg_info
creating tfx_user_code_ModelTrainer.egg-info
writing tfx_user_code_ModelTrainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_ModelTrainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_ModelTrainer.egg-info/top_lev

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


In [17]:
PIPELINES_STORE = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/compiled_pipelines/"
!gsutil cp {pipeline_definition_file} {PIPELINES_STORE}

Copying file://chicago-taxi-tips-classifier-v01-train-pipeline.json [Content-Type=application/json]...
/ [1 files][ 26.2 KiB/ 26.2 KiB]                                                
Operation completed over 1 objects/26.2 KiB.                                     


### Submit run to Vertex Pipelines

In [18]:
from kfp.v2.google.client import AIPlatformClient

pipeline_client = AIPlatformClient(
    project_id=PROJECT, region=REGION)
                 
job = pipeline_client.create_run_from_job_spec(
    job_spec_path=pipeline_definition_file,
    parameter_values={
        'learning_rate': 0.003,
        'batch_size': 512,
        'hidden_units': '128,128',
        'num_epochs': 30,
    }
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/google/client/client.py:173: FutureWarning: AIPlatformClient will be deprecated in v2.0.0. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


### Extracting pipeline runs metadata

In [19]:
from google.cloud import aiplatform as vertex_ai

pipeline_df = vertex_ai.get_pipeline_df(PIPELINE_NAME)
pipeline_df = pipeline_df[pipeline_df.pipeline_name == PIPELINE_NAME]
pipeline_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
pipeline_name,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline
run_name,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...
param.input:learning_rate,0.003,0.0015,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003,0.003
param.input:num_epochs,30,3,30,1,1,1,1,30,30,30,30,30,30,30,30
param.input:batch_size,512,512,512,512,512,512,512,512,512,512,512,512,512,512,512
param.input:hidden_units,"128,128","256,126","128,128","128,128","128,128","128,128","128,128","128,128","128,128","128,128","128,128","128,128","128,128","128,128","128,128"


## 3. Execute the pipeline deployment CI/CD steps in Cloud Build

The CI/CD routine is defined in the [pipeline-deployment.yaml](pipeline-deployment.yaml) file, and consists of the following steps:
1. Clone the repository to the build environment.
2. Run unit tests.
3. Run a local e2e test of the pipeline.
4. Build the ML container image for pipeline steps.
5. Compile the pipeline.
6. Upload the pipeline to Cloud Storage.

### Build CI/CD container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying the pipeline will be executed.

In [20]:
!echo $CICD_IMAGE_URI

gcr.io/mwpmltr/cicd:latest


In [ ]:
!gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m --machine-type=e2-highcpu-8

Creating temporary tarball archive of 10 file(s) totalling 22.5 KiB before compression.
Uploading tarball of [build/.] to [gs://mwpmltr_cloudbuild/source/1662854044.883605-72513495fd5b49f78b5882ad22a29dc2.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/mwpmltr/locations/global/builds/978f9be9-0487-4aa6-bb72-de7f5421ed6b].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/978f9be9-0487-4aa6-bb72-de7f5421ed6b?project=55590906972].
Copying gs://mwpmltr_cloudbuild/source/1662854044.883605-72513495fd5b49f78b5882ad22a29dc2.tgz#1662854045111602...
/ [1 files][  3.4 KiB/  3.4 KiB]                                                
Operation completed over 1 objects/3.4 KiB.
BUILD
Already have image (with digest): gcr.io/cloud-builders/docker
Sending build context to Docker daemon  32.26kB
Step 1/4 : FROM gcr.io/tfx-oss-public/tfx:1.8.0
1.8.0: Pulling from tfx-oss-public/tfx
d5fd17ec1767: Pulling fs layer
086b79b77a03: Pulling fs layer
4698168f5888: Pulling fs 

### Run CI/CD from pipeline deployment using Cloud Build

In [ ]:
REPO_URL = "https://github.com/GoogleCloudPlatform/mlops-with-vertex-ai.git" # Change to your github repo.
BRANCH = "main"

GCS_LOCATION = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/"
TEST_GCS_LOCATION = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/e2e_tests"
CI_TRAIN_LIMIT = 1000
CI_TEST_LIMIT = 100
CI_UPLOAD_MODEL = 0
CI_ACCURACY_THRESHOLD = 0.1
BEAM_RUNNER = "DataflowRunner"
TRAINING_RUNNER = "vertex"
VERSION = 'tfx-1.8'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'
PIPELINES_STORE = os.path.join(GCS_LOCATION, "compiled_pipelines")

TFX_IMAGE_URI = f"gcr.io/{PROJECT}/{DATASET_DISPLAY_NAME}:{VERSION}"

SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_GCS_LOCATION={GCS_LOCATION},\
_TEST_GCS_LOCATION={TEST_GCS_LOCATION},\
_BQ_LOCATION={BQ_LOCATION},\
_BQ_DATASET_NAME={BQ_DATASET_NAME},\
_BQ_TABLE_NAME={BQ_TABLE_NAME},\
_DATASET_DISPLAY_NAME={DATASET_DISPLAY_NAME},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_CI_TRAIN_LIMIT={CI_TRAIN_LIMIT},\
_CI_TEST_LIMIT={CI_TEST_LIMIT},\
_CI_UPLOAD_MODEL={CI_UPLOAD_MODEL},\
_CI_ACCURACY_THRESHOLD={CI_ACCURACY_THRESHOLD},\
_BEAM_RUNNER={BEAM_RUNNER},\
_TRAINING_RUNNER={TRAINING_RUNNER},\
_TFX_IMAGE_URI={TFX_IMAGE_URI},\
_PIPELINE_NAME={PIPELINE_NAME},\
_PIPELINES_STORE={PIPELINES_STORE}\
"""

!echo $SUBSTITUTIONS

In [ ]:
!gcloud builds submit --no-source --timeout=60m --config build/pipeline-deployment.yaml --substitutions {SUBSTITUTIONS} --machine-type=e2-highcpu-8